In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from safetensors.torch import save_file
from datasets import load_dataset
from dotenv import load_dotenv
import os
from typing import List
load_dotenv()


device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(device)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-405B-FP8", token=os.getenv("HUGGINGFACE"))
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-405B-FP8", token=os.getenv("HUGGINGFACE")).to(device)
model.eval()

In [ ]:
ds = load_dataset("openai/gsm8k", "main", split="train")
questions = ds["question"]

In [ ]:
def get_logits(inputs : str) -> torch.tensor:
    inputs = tokenizer(inputs, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        return outputs.logits

In [ ]:
def save_logits(logits : List[torch.tensor], ) -> None:
    data = {}
    for i, logit in enumerate(logits):
        data[f"Question {i+1}"] = logit.cpu()

    save_file(data, "llama-3.1-405b-gsm8k-base-tensors.safetensors")

In [ ]:
# to be used later on to load back the logits

from safetensors.torch import safe_open

def load_list_of_logits_safetensor(file_path):
    # Open the safetensor file
    with safe_open(file_path, framework="pt") as f:
        logits_list = []
        for key in f.keys():
            logits_list.append(f.get_tensor(key))
    
    return logits_list


In [ ]:
logits = [get_logits(question) for question in questions]
save_logits(logits)

RuntimeError: Failed to import transformers.integrations.fbgemm_fp8 because of the following error (look up to see its traceback):
libcudart.so.12: cannot open shared object file: No such file or directory

linux-vdso.so.1 (0x00007ffe239fe000)
        libtorch.so => not found (wtf??????)
        libc10.so => not found   
        libcudart.so.12 => /usr/local/cuda/targets/x86_64-linux/lib/libcudart.so.12 (0x000073e204e00000) (why did this say not found???)
        libc10_cuda.so => not found (issue)
        libnvidia-ml.so.1 => /usr/lib/x86_64-linux-gnu/libnvidia-ml.so.1 (0x000073e203c00000)
        libtorch_cpu.so => not found
        libtorch_cuda.so => not found (massive issue)
        librt.so.1 => /usr/lib/x86_64-linux-gnu/librt.so.1 (0x000073e20513a000)
        libpthread.so.0 => /usr/lib/x86_64-linux-gnu/libpthread.so.0 (0x000073e205135000)
        libstdc++.so.6 => /usr/lib/x86_64-linux-gnu/libstdc++.so.6 (0x000073e2039d4000)
        libm.so.6 => /usr/lib/x86_64-linux-gnu/libm.so.6 (0x000073e204d19000)
        libgcc_s.so.1 => /usr/lib/x86_64-linux-gnu/libgcc_s.so.1 (0x000073e205113000)
        libc.so.6 => /usr/lib/x86_64-linux-gnu/libc.so.6 (0x000073e2037ab000)
        /lib64/ld-linux-x86-64.so.2 (0x000073e25e59b000)
        libdl.so.2 => /usr/lib/x86_64-linux-gnu/libdl.so.2 (0x000073e20510e000)